# ETL

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import requests

warnings.filterwarnings('ignore')
sns.set()

Accesos a internet
- 'Accesos_provincias.csv'
- 'Internet_Accesos-por-tecnologia.csv',
- 'Internet_Accesos-por-tecnologia_provincias.csv',
- 'Internet_Accesos-por-velocidad.csv',
- 'Internet_Accesos-por-velocidad_provincias.csv'
- 'Internet_BAF.csv',
- 'internet_fijo_tecnologia_localidades.csv'

Velocidad de internet
- 'historico_velocidad_internet.csv',
- 'historico_velocidad_internet_provincias.csv'
- 'internet_vel_bajada_localidades.csv'
- 'velocidad_bajada_provincias.csv'

Penetraciones a internet
- 'Internet_Penetracion.csv',
- 'Internet_Penetracion_hogares.csv'

Listado de localidades de todo el pais con internet
- 'listado_localidades.csv'

Ingresos trimestrales
- 'Internet_Ingresos_trimestrales.csv'

In [ ]:
def get_url(url):
    api_key = '7rzbGLPyLBX5oegA957IyPUBmjp7ixTlxO04Ql0t'
    url = url.replace('YOUR_API_KEY',api_key)
    return url

In [3]:
df = pd.read_csv('original_datasets/Accesos_provincias.csv')
df.head()

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,3,Buenos Aires,4.715.469,6.199,4.721.668
1,2022,3,Capital Federal,1.545.534,2.145,1.547.679
2,2022,3,Catamarca,70.292,1,70.293
3,2022,3,Chaco,144.141,5,144.146
4,2022,3,Chubut,164.874,904,165.778


In [4]:
# Creo una columna que represente el 'id' para poder hacer un merge con otro df
df['id'] = df.Año.astype(str) + df.Trimestre.astype(str) + df.Provincia.astype(str)
df.head()

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total,id
0,2022,3,Buenos Aires,4.715.469,6.199,4.721.668,20223Buenos Aires
1,2022,3,Capital Federal,1.545.534,2.145,1.547.679,20223Capital Federal
2,2022,3,Catamarca,70.292,1,70.293,20223Catamarca
3,2022,3,Chaco,144.141,5,144.146,20223Chaco
4,2022,3,Chubut,164.874,904,165.778,20223Chubut


Hago lo mismo con el otro dataset

In [5]:
df2 = pd.read_csv('original_datasets\Internet_Accesos-por-tecnologia_provincias.csv')
df2.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,3,Buenos Aires,339.648,2.748.325,1.436.433,126.846,70.416,4.721.668
1,2022,3,Capital Federal,140.791,1.240.125,129.218,5.758,31.787,1.547.679
2,2022,3,Catamarca,10.010,10.495,46.224,1.329,2.235,70.293
3,2022,3,Chaco,27.164,61.800,44.645,8.178,2.359,144.146
4,2022,3,Chubut,45.377,72.212,9.574,29.784,8.831,165.778


In [6]:
df2['id'] = df2.Año.astype(str) + df2.Trimestre.astype(str) + df2.Provincia.astype(str)
df2.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,id
0,2022,3,Buenos Aires,339.648,2.748.325,1.436.433,126.846,70.416,4.721.668,20223Buenos Aires
1,2022,3,Capital Federal,140.791,1.240.125,129.218,5.758,31.787,1.547.679,20223Capital Federal
2,2022,3,Catamarca,10.010,10.495,46.224,1.329,2.235,70.293,20223Catamarca
3,2022,3,Chaco,27.164,61.800,44.645,8.178,2.359,144.146,20223Chaco
4,2022,3,Chubut,45.377,72.212,9.574,29.784,8.831,165.778,20223Chubut


Hago el merge

In [7]:
# Selecciono solo las columnas que quiero para le merge
df_merged = pd.merge(df2, df[['Banda ancha fija','Dial up','Total','id']], how='left', on='id')
df_merged.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total_x,id,Banda ancha fija,Dial up,Total_y
0,2022,3,Buenos Aires,339.648,2.748.325,1.436.433,126.846,70.416,4.721.668,20223Buenos Aires,4.715.469,6.199,4.721.668
1,2022,3,Capital Federal,140.791,1.240.125,129.218,5.758,31.787,1.547.679,20223Capital Federal,1.545.534,2.145,1.547.679
2,2022,3,Catamarca,10.010,10.495,46.224,1.329,2.235,70.293,20223Catamarca,70.292,1,70.293
3,2022,3,Chaco,27.164,61.800,44.645,8.178,2.359,144.146,20223Chaco,144.141,5,144.146
4,2022,3,Chubut,45.377,72.212,9.574,29.784,8.831,165.778,20223Chubut,164.874,904,165.778


Transformo los valores a numeros

In [8]:
df_merged_columns = ['Año',
 'Trimestre',
 'ADSL',
 'Cablemodem',
 'Fibra óptica',
 'Wireless',
 'Otros',
 'Total_x',
 'Banda ancha fija',
 'Dial up',
 'Total_y'
 ]

for column in df_merged_columns:
    column_name = df_merged[column].astype(str).str.replace('.','').replace('-','').replace('*','').replace(' ','')
    column_name = column_name.str.strip(' *')
    df_merged[column] = pd.to_numeric(column_name, errors = 'coerce')

In [9]:
df_merged.sample(3)

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total_x,id,Banda ancha fija,Dial up,Total_y
594,2016.0,3.0,San Luis,11432.0,1113.0,354.0,230.0,3710.0,13293.0,20163San Luis,12978.0,315.0,13293.0
527,2017.0,2.0,Tucumán,149013.0,5223.0,116.0,140.0,450.0,154411.0,20172Tucumán,154374.0,37.0,154411.0
600,2016.0,2.0,Buenos Aires,1579448.0,1305949.0,113070.0,21163.0,25661.0,3045291.0,20162Buenos Aires,3032776.0,12515.0,3045291.0


Como me di cuenta que el valor de Dial up esta dentro del valor de Otros, decido restarle el valor para asi conservar la columna y para no alterar al Total

In [10]:
df_merged['Otros'] = df_merged['Otros'] - df_merged['Dial up']

In [11]:
# Elimino las columnas que estan de mas
df_merged.drop(columns={'Banda ancha fija','Total_y'}, inplace=True)

In [12]:
# Ordeno las columnas a mi parecer
df_merged = df_merged[['id','Año', 'Trimestre', 'Provincia', 'ADSL', 'Cablemodem', 'Fibra óptica',
       'Wireless','Dial up','Otros','Total_x']]

# Renombro a la columna que representa al Total
df_merged.rename(columns={'Total_x':'Total'}, inplace=True)

In [16]:
df_merged.sample(5)

,id,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Dial up,Otros,Total
268,20194Chubut,2019.0,4.0,Chubut,47045.0,26977.0,1324.0,15267.0,810.0,4679.0,96102.0
351,2019 *1 *Río Negro,2019.0,1.0,Río Negro,68377.0,34124.0,3994.0,9736.0,NaN,NaN,121088.0
204,20203Mendoza,2020.0,3.0,Mendoza,91908.0,64672.0,32817.0,17523.0,615.0,8065.0,207788.0
23,20223Tucumán,2022.0,3.0,Tucumán,77738.0,18518.0,162345.0,3997.0,30.0,5160.0,263117.0
369,20184Jujuy,2018.0,4.0,Jujuy,33248.0,12484.0,153.0,6400.0,1.0,43607.0,90133.0


In [18]:
df_merged.sample(3)

,id,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Dial up,Otros,Total
372,20184Mendoza,2018.0,4.0,Mendoza,148524.0,40205.0,5665.0,10363.0,1322.0,225.0,206304.0
303,2019 *3 *Río Negro,2019.0,3.0,Río Negro,67478.0,37425.0,9739.0,10806.0,NaN,NaN,144295.0
593,20163San Juan,2016.0,3.0,San Juan,48670.0,451.0,770.0,1174.0,620.0,8410.0,51968.0


In [19]:
df_merged.to_csv('clean_datasets/accesos_provincias_trimestral.csv', index=False)

---

## Velocidad por provincia

In [22]:
df1 = pd.read_csv('original_datasets\historico_velocidad_internet_provincias.csv')
df1.head()

,Año,Trimestre,Provincia,Mbps (Media de bajada),Unnamed: 4,Unnamed: 5
0,2022,3,Buenos Aires,70,NaN,NaN
1,2022,3,Capital Federal,101,NaN,NaN
2,2022,3,Catamarca,61,NaN,NaN
3,2022,3,Chaco,53,NaN,NaN
4,2022,3,Chubut,16,NaN,NaN


In [23]:
# Transformo la columna de Mbps a valores numericos
df1['Mbps (Media de bajada)'] = pd.to_numeric(df1['Mbps (Media de bajada)'])

# Elimino las columnas con todos los valores nulos
df1.drop(columns={'Unnamed: 4', 'Unnamed: 5'}, inplace= True)

In [24]:
df1.sample(3)

,Año,Trimestre,Provincia,Mbps (Media de bajada)
560,2016,4,Formosa,6
812,2014,2,Santa Fe,3
354,2019,1,San Luis,9


In [30]:
df1.to_csv('clean_datasets/mbps_provincias_trimestral', index=False)

---

### Dataset `Internet_Accesos-por-velocidad`

In [28]:
df2 = pd.read_csv('original_datasets\Internet_Accesos-por-velocidad.csv')
df2.sample(3)

,Año,Trimestre,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Total,Unnamed: 11
18,2018,1,7.309,215.406,3.272.724,976.956,2.068.087,862.010,397.977,236.584,8.037.053,NaN
2,2022,1,34.890,104.840,1.263.273,1.209.148,967.508,509.830,6.336.187,198.333,10.624.009,NaN
7,2020,4,39.510,42.185,2.622.638,1.073.875,786.595,582.420,4.239.237,185.102,9.571.562,NaN


In [29]:
# Elimino la columna Unnamed: 11
df2.drop(columns={'Unnamed: 11'}, inplace = True)
df2.sample(3)

,Año,Trimestre,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Total
12,2019,3,241.713,80.599,3.309.137,1.026.859,806.466,1.141.545,2.314.898,243.467,9.164.684
28,2015,3,35.030,455.777,5.087.802,701.187,539.414,13.101,20.677,0.000,6.852.988
3,2021,4,41.262,28.521,1.413.208,1.245.333,976.539,558.358,6.032.322,194.251,10.489.794


In [31]:
df2.to_csv('clean_datasets/velocidad_pais_trimestral.csv', index=False)

---

### Datasets `Internet_Penetracion_hogares` y `Internet_Penetracion`

In [38]:
df = pd.read_csv('original_datasets\Internet_Penetracion_hogares.csv')
df.head()

,Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo
0,2022,3,"76,64","23,95",Jul-Sept 2022
1,2022,2,"75,97","23,72",Abr-Jun 2022
2,2022,1,"73,88","23,05",Ene-Mar 2022
3,2021,4,"73,18","22,81",Oct-Dic 2021
4,2021,3,"70,58","21,98",Jul-Sept 2021


In [39]:
df['Accesos por cada 100 hogares'] = df['Accesos por cada 100 hogares'].str.replace(',','.')
df['Accesos por cada 100 hab'] = df['Accesos por cada 100 hab'].str.replace(',','.')

In [41]:
df.to_csv('clean_datasets/acceso_por_hogares_habitantes_pais_trimestral.csv', index= False)

In [5]:
df2 = pd.read_csv('original_datasets\Internet_Penetracion.csv')

# Transformo las columnas numericas a valores numericos
df2['Año'] = pd.to_numeric(df2['Año'])
df2['Trimestre'] = pd.to_numeric(df2['Trimestre'])
df2['Accesos por cada 100 hogares'] = pd.to_numeric(df2['Accesos por cada 100 hogares'].astype(str).str.replace(',','.')).astype(float)

In [6]:
df2

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2022,3,Buenos Aires,78.11
1,2022,3,Capital Federal,122.28
2,2022,3,Catamarca,65.33
3,2022,3,Chaco,43.86
4,2022,3,Chubut,84.38
...,...,...,...,...
835,2014,1,Santa Cruz,28.94
836,2014,1,Santa Fe,47.85
837,2014,1,Santiago Del Estero,16.31
838,2014,1,Tierra Del Fuego,63.97


In [7]:
df2.to_csv('clean_datasets/accesos_por_hogares_provincias_trimestral.csv', index=False)

---

### Dataset `listado_localidades`

In [53]:
df3 = pd.read_csv('original_datasets\listado_localidades.csv')
df3.sample(3)

,Unnamed: 0,Partido,Localidad,ADSL,Unnamed: 4,DIALUP,FIBRAOPTICA,4G,Unnamed: 8,TELEFONIAFIJA,Unnamed: 10,SATELITAL
1637,NaN,General Roca,Barrio La Ribera - Barrio APYCAR,--,NaN,--,--,NaN,NaN,--,SI,NaN
4230,CHACO,NaN,General José de San Martín,SI,SI,NaN,SI,SI,SI,NaN,SI,NaN
4048,CATAMARCA,NaN,Los Altos,--,--,NaN,--,SI,SI,NaN,SI,SI


In [54]:
# Elimino las columnas que sobran
df3.drop(columns={'Unnamed: 4','Unnamed: 8','Unnamed: 10'}, inplace=True)

df3.replace('--','No', inplace=True)
df3.replace('SI','Si', inplace=True)
df3.rename(columns={'Unnamed: 0':'Provincia'}, inplace=True)

In [55]:
df3.sample(4)

,Provincia,Partido,Localidad,ADSL,DIALUP,FIBRAOPTICA,4G,TELEFONIAFIJA,SATELITAL
2236,LA RIOJA,NaN,Antinaco,No,NaN,No,No,NaN,No
4117,LA RIOJA,San Blas de los Sauces,San Blas,NaN,No,No,NaN,No,No
1480,NaN,Coronel Pringles,Intihuasi,No,No,No,No,No,No
3619,CATAMARCA,NaN,Las Juntas,No,NaN,No,Si,NaN,No


In [45]:
df3['FIBRAOPTICA'].value_counts()

No    1907
Si     961
Name: FIBRAOPTICA, dtype: int64

In [46]:
df3['4G'].value_counts()

Si    1723
No    1139
Name: 4G, dtype: int64

In [56]:
df3.to_csv('clean_datasets/servicios_localidades_partido.csv', index = False)

---

### Dataset `Internet_Ingresos_trimestrales.csv`

In [91]:
df = pd.read_csv('original_datasets\Internet_Ingresos_trimestrales.csv')
df.head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,3,67.055.930,Jul-Sept 2022
1,2022,2,60.335.724,Abr-Jun 2022
2,2022,1,55.589.997,Ene-Mar 2022
3,2021,4,45.467.887,Oct-Dic 2021
4,2021,3,42.999.944,Jul-Sept 2021


In [92]:
# Transformo la columna en un valor numerico con decimales
df['Ingresos (miles de pesos)'] = df['Ingresos (miles de pesos)'].astype(str).str.slice_replace(2,3, '')

Veo como quedan los cambios

In [77]:
df

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,3,67055.930,Jul-Sept 2022
1,2022,2,60335.724,Abr-Jun 2022
2,2022,1,55589.997,Ene-Mar 2022
3,2021,4,45467.887,Oct-Dic 2021
4,2021,3,42999.944,Jul-Sept 2021
5,2021,2,38239.667,Abr-Jun 2021
6,2021,1,36676.371,Ene-Mar 2021
7,2020,4,33539.703,Oct-Dic 2020
8,2020,3,31997.445,Jul-Sept 2020
9,2020,2,32102.476,Abr-Jun 2020


A partir de la fila 21 las transformaciones fallan, asi que voy a dividir el df en dos y despues hago un pd.concat()

In [93]:
df1 = df.iloc[:21]
df2 = df.iloc[21:]

In [94]:
df2['Ingresos (miles de pesos)'] = df2['Ingresos (miles de pesos)'].astype(str).str.replace('.','').str.slice_replace(-2,-1,'.')

In [95]:
df = pd.concat([df1, df2], ignore_index=True)
df

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,3,67055.930,Jul-Sept 2022
1,2022,2,60335.724,Abr-Jun 2022
2,2022,1,55589.997,Ene-Mar 2022
3,2021,4,45467.887,Oct-Dic 2021
4,2021,3,42999.944,Jul-Sept 2021
5,2021,2,38239.667,Abr-Jun 2021
6,2021,1,36676.371,Ene-Mar 2021
7,2020,4,33539.703,Oct-Dic 2020
8,2020,3,31997.445,Jul-Sept 2020
9,2020,2,32102.476,Abr-Jun 2020


In [96]:
df['Ingresos (miles de pesos)'] = pd.to_numeric(df['Ingresos (miles de pesos)']).astype(float)

In [97]:
df.to_csv('clean_datasets/ingresos_internet_trimestral.csv', index = False)

---

## Velocidad

In [78]:
df = pd.read_csv('original_datasets/velocidad_bajada_provincias.csv')
df.head()

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",...,70 Mbps,71 Mbps,75 Mbps,77 Mbps,78 Mbps,80 Mbps,81 Mbps,83 Mbps,82 Mbps,86 Mbps
0,2022.0,3.0,Buenos Aires,"65,821",19,- 0,- 0,"29,774",192,"4,611",...,"3,163",- 0,"98,387",- 0,2,223,- 0,1,- 0,- 0
1,2022.0,3.0,Capital Federal,"105,477",31,- 0,- 0,451,35,"1,307",...,3,- 0,"30,482",- 0,- 0,8,- 0,- 0,- 0,- 0
2,2022.0,3.0,Catamarca,"2,208",- 0,- 0,- 0,34,37,- 0,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
3,2022.0,3.0,Chaco,"3,711",- 0,225,111,234,2,- 0,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
4,2022.0,3.0,Chubut,"20,024",8,- 0,- 0,69,32,836,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0


In [79]:
for column in df.columns:
    df[column] = df[column].astype(str).str.replace('- 0','0').str.replace(',','')

In [80]:
# Elimino la ultima fila ya que eran todos valores nulos en todas las columnas
df = df.drop(df.index.max())

In [85]:
df['Año'] = pd.to_numeric(df['Año']).astype(int)
df['Trimestre'] = pd.to_numeric(df['Trimestre']).astype(int)

In [87]:
df.sample(5)

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",...,70 Mbps,71 Mbps,75 Mbps,77 Mbps,78 Mbps,80 Mbps,81 Mbps,83 Mbps,82 Mbps,86 Mbps
27,2022,2,Chaco,3708,0,225,111,234,2,0,...,0,0,0,0,0,0,0,0,0,0
66,2022,1,San Luis,28589,0,0,0,60,1,222,...,0,0,0,0,0,0,0,0,0,0
344,2019,1,Formosa,0,0,0,0,1110,0,191,...,0,0,0,0,0,0,0,0,0,0
270,2019,4,Corrientes,4995,0,0,0,10,62,0,...,0,0,0,0,0,0,0,0,0,0
380,2018,4,Santa Fe,28450,0,0,0,335,0,2928,...,0,0,0,0,0,0,0,0,0,0


In [88]:
df.to_csv('clean_datasets/velocidad_bajada_provincias.csv', index= False)

### Dataset `Internet_BAF`

In [90]:
df = pd.read_csv('original_datasets\Internet_BAF.csv')
df.head()

,Año,Trimestre,Banda ancha fija,Dial up,Total,Periodo
0,2022,3,11.078.691,12.437,11.091.128,Jul-Sept 2022
1,2022,2,10.946.248,12.436,10.958.684,Abr-Jun 2022
2,2022,1,10.611.390,12.619,10.624.009,Ene-Mar 2022
3,2021,4,10.476.933,12.861,10.489.794,Oct-Dic 2021
4,2021,3,10.075.184,10.357,10.085.541,Jul-Sept 2021


In [99]:
df2 = pd.read_csv('clean_datasets/accesos_provincias_trimestral.csv')
df2.groupby(by=['Año','Trimestre']).sum().sort_values(by=['Año','Trimestre'], ascending=False).head()

ADSL  Cablemodem  Fibra óptica  Wireless  Dial up  \
Año    Trimestre                                                           
2022.0 3.0        1395277.0   6031970.0     2871541.0  468514.0  12437.0   
       2.0        1468333.0   5979214.0     2723285.0  507067.0  12436.0   
       1.0        1533240.0   6073426.0     2219533.0  541575.0  12619.0   
2021.0 4.0        1657615.0   5984240.0     2072236.0  460719.0  12861.0   
       3.0        1950631.0   5826257.0     1566048.0  437830.0  10357.0   

                     Otros       Total  
Año    Trimestre                        
2022.0 3.0        230743.0  11091128.0  
       2.0        200628.0  10958684.0  
       1.0        252796.0  10624009.0  
2021.0 4.0        241407.0  10489794.0  
       3.0        211823.0  10085541.0

Como lo suponia, el dataset de `Internet_BAF` contiene la misma información que el dataset `accesos_provincias_trimestral` pero agrupado por trimestre. Por esto, considero que no es necesario tenerlo ya que podria utilizar el otro dataframe, asi qeu lo elimino

---

### Dataset `internet_fijo_tecnologia_localidades`

In [110]:
df = pd.read_csv('original_datasets\internet_fijo_tecnologia_localidades.csv')
df.sample(5)


,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general,Unnamed: 13
1346,CORRIENTES,ItatÍ,Ramada Paso,18077020,- 0,- 0,- 0,- 0,- 0,- 0,- 0,3,3,NaN
372,BUENOS AIRES,La Plata,Arana,6441030,- 0,- 0,- 0,- 0,- 0,- 0,- 0,129,129,NaN
2192,RIO NEGRO,General Roca,Catriel,62042360,- 0,- 0,- 0,507,2.619,2,- 0,19,3.147,NaN
2614,SANTA FE,Castellanos,Angélica,82021020,600,- 0,- 0,- 0,- 0,- 0,- 0,9,609,NaN
2641,SANTA FE,Castellanos,San Vicente,82021340,770,1.334,- 0,- 0,- 0,- 0,- 0,42,2.146,NaN


In [111]:
# Elimino la columna con valores nulos
df = df.drop(columns={'Unnamed: 13'})

# Reemplazo los valores - 0 a 0
columns = ['ADSL', 'CABLEMODEM',
       'DIAL UP', 'FIBRA OPTICA', 'OTROS', 'SATELITAL', 'WIMAX', 'WIRELESS',
       'Total general']

for c in columns:
    df[c] = df[c].astype(str).str.replace('- 0','0').str.replace('.','')

In [116]:
df.sample(5)

,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general
2099,NEUQUEN,Confluencia,Arroyito,58035010,0,0,0,0,0,0,0,22,22
1017,CORDOBA,General San Martín,Ticino,14042140,0,663,0,6,0,2,0,0,671
1586,FORMOSA,Pilcomayo,Puerto Pilcomayo,34049050,0,0,0,0,0,0,0,3,3
151,BUENOS AIRES,Colón,Colón,6175010,4143,1588,0,1794,0,0,0,20,7545
2937,SANTIAGO DEL ESTERO,Aguirre,Malbrán,86007030,13,0,0,0,0,1,0,0,14


In [117]:
df.to_csv('clean_datasets/internet_fijo_localidades_partidos.csv', index= False)

---

### Dataset `historico_velocidad_internet`

In [120]:
df = pd.read_csv('original_datasets\historico_velocidad_internet.csv')
df.head()

,Año,Trimestre,Mbps (Media de bajada),Trimestre.1
0,2022,3,"62,46",Jul-Sept 2022
1,2022,2,"58,44",Abr-Jun 2022
2,2022,1,"55,11",Ene-Mar 2022
3,2021,4,"52,34",Oct-Dic 2021
4,2021,3,"48,46",Jul-Sept 2021


In [123]:
df['Mbps (Media de bajada)'] = pd.to_numeric(df['Mbps (Media de bajada)'].astype(str).str.replace(',','.')).astype(float)

In [125]:
df.to_csv('clean_datasets/mbps_pais_trimestral.csv', index=False)

---

In [127]:
df = pd.read_csv('original_datasets\internet_vel_bajada_localidades.csv')
df

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 mbps","0,375 mbps","0,5 mbps","0,512 mbps","0,625 mbps",...,75 mbps,78 mbps,80 mbps,82 mbps,83 mbps,85 mbps,90 mbps,92 mbps,95 mbps,100 mbps
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,NaN,NaN,NaN,2,NaN,NaN,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,6854010,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BUENOS AIRES,25 de Mayo,Del Valle,6854020,NaN,NaN,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUENOS AIRES,25 de Mayo,Ernestina,6854030,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,TUCUMAN,Tafí Viejo,Villa Mariano Moreno - El Colmenar,90105100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,304
3094,TUCUMAN,Trancas,San Pedro de Colalao,90112020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3095,TUCUMAN,Trancas,Villa de Trancas,90112030,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3096,TUCUMAN,Yerba Buena,Villa Carmela,90119020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
columns = ['Otros',
       '0,256 mbps', '0,375 mbps', '0,5 mbps', '0,512 mbps', '0,625 mbps',
       '0,75 mbps', '1 mbps', '1,25 mbps', '1,5 mbps', '2 mbps', '2,2 mbps',
       '2,5 mbps', '3 mbps', '3,3 mbps', '3,5 mbps', '4 mbps', '4,5 mbps',
       '5 mbps', '6 mbps', '6,4 mbps', '7 mbps', '7,5 mbps', '8 mbps',
       '9 mbps', '10 mbps', '11 mbps', '12 mbps', '13 mbps', '14 mbps',
       '15 mbps', '16 mbps', '17 mbps', '18 mbps', '19 mbps', '20 mbps',
       '21 mbps', '22 mbps', '23 mbps', '24 mbps', '25 mbps', '25,1 mbps',
       '25,11 mbps', '25,5 mbps', '26 mbps', '30 mbps', '31 mbps', '32 mbps',
       '34 mbps', '35 mbps', '36 mbps', '38 mbps', '39 mbps', '40 mbps',
       '41 mbps', '45 mbps', '46 mbps', '49 mbps', '50 mbps', '55 mbps',
       '58 mbps', '59 mbps', '60 mbps', '61 mbps', '62 mbps', '64 mbps',
       '65 mbps', '66 mbps', '70 mbps', '75 mbps', '78 mbps', '80 mbps',
       '82 mbps', '83 mbps', '85 mbps', '90 mbps', '92 mbps', '95 mbps',
       '100 mbps']

for c in columns:
    df[c] = pd.to_numeric(df[c].astype(str).fillna('0').str.replace(',','.')).astype(int)

In [139]:
df.sample(4)

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 mbps","0,375 mbps","0,5 mbps","0,512 mbps","0,625 mbps",...,75 mbps,78 mbps,80 mbps,82 mbps,83 mbps,85 mbps,90 mbps,92 mbps,95 mbps,100 mbps
1073,CORDOBA,Presidente Roque Sáenz Peña,Laboulaye,14084030,229,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2438,SAN LUIS,Chacabuco,El Algarrobal,74028020,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2144,RIO NEGRO,Avellaneda,Darwin,62014060,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1567,FORMOSA,Pilcomayo,Laguna Naick-Neck,34049030,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
df.to_csv('clean_datasets/velocidad_bajada_localidades.csv', index=False)

---

### Dataset `Internet_Accesos-por-velocidad_provincias`

In [143]:
df = pd.read_csv('original_datasets/Internet_Accesos-por-velocidad_provincias.csv')
df.head()

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,3,Buenos Aires,29.985,27.709,290.315,297.915,267.044,124.190,3.618.689,65.821,4.721.668
1,2022,3,Capital Federal,517.000,5.742,34.371,67.829,51.946,28.692,1.253.105,105.477,1.547.679
2,2022,3,Catamarca,71.000,384.000,3.107,5.389,5.099,3.737,50.298,2.208,70.293
3,2022,3,Chaco,461.000,987.000,16.782,18.938,8.049,15.828,79.390,3.711,144.146
4,2022,3,Chubut,109.000,1.444,45.707,30.940,34.682,15.309,17.563,20.024,165.778


In [145]:
df

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,3,Buenos Aires,29985,27709,290315,297915,267044,12419,3618689,65.821,4.721.668
1,2022,3,Capital Federal,5170,5742,34371,67829,51946,28692,1253105,105.477,1.547.679
2,2022,3,Catamarca,710,3840,3107,5389,5099,3737,50298,2.208,70.293
3,2022,3,Chaco,4610,9870,16782,18938,8049,15828,79390,3.711,144.146
4,2022,3,Chubut,1090,1444,45707,3094,34682,15309,17563,20.024,165.778
...,...,...,...,...,...,...,...,...,...,...,...,...
835,2014,1,Santa Cruz,1610,1625,24972,10,10,0,0,0.000,26.760
836,2014,1,Santa Fe,8456,124468,345225,20328,6845,230,668,0.000,506.013
837,2014,1,Santiago Del Estero,1234,10531,22817,2422,1090,0,0,0.000,37.113
838,2014,1,Tierra Del Fuego,120,6070,30902,60,0,0,0,0.000,31.527


In [147]:
columns = ['HASTA 512 kbps',
       '+ 512 Kbps - 1 Mbps', '+ 1 Mbps - 6 Mbps', '+ 6 Mbps - 10 Mbps',
       '+ 10 Mbps - 20 Mbps', '+ 20 Mbps - 30 Mbps', '+ 30 Mbps', 'OTROS',
       'Total']

for c in columns:
    df[c] = pd.to_numeric(df[c].fillna('0').astype(str).str.replace('.','')).astype(int)

In [149]:
df.sample(3)

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
191,2020,4,Tucumán,290,970,44095,54334,32155,4105,93594,268,231089
634,2016,1,La Pampa,4260,11065,52117,150,780,710,11,0,63783
719,2015,2,Tucumán,1070,25178,83462,26966,4502,4550,0,0,140670


In [150]:
df.to_csv('clean_datasets/velocidad_provincias_trimestral.csv', index= False)